In [ ]:
%load_ext autoreload

In [ ]:
import pandas as pd
from os.path import join

%autoreload
from base import read_sf_xd_segments
from xd_diff import read_replace_csv, xd_segments_added, xd_segments_removed


In [ ]:
# variables you have to set (also set replace_diff_version 2 cells down)

conflation_dir = r"Q:\CMP\LOS Monitoring 2023\Network_Conflation\v2301"
current_version = "2301"
previous_version = "2202"

network_conflation_correspondences_filepath = join(
    conflation_dir, "CMP_Segment_INRIX_Links_Correspondence_2301_Manual.csv"
)

In [ ]:
replace_df = read_replace_csv(current_version)
replace_df.columns

In [ ]:
# variables you have to set: replace_df does not necessary do a diff between
# the current version vs the previous version; sometimes it compares the current
# version to older versions, so you have to set `replace_diff_version`
# manually here, based on the results of `replace_df.columns` above
replace_diff_version = "2201"

In [ ]:
current_xd_segments_gdf = read_sf_xd_segments(current_version)
previous_xd_segments_gdf = read_sf_xd_segments(previous_version)
summary = {}

In [ ]:
# INRIX XD segments added in this map release update:
xd_segments_added_gdf = xd_segments_added(
    current_xd_segments_gdf, current_version
)
summary["num_segments_added"] = len(xd_segments_added_gdf)
xd_segments_added_gdf

In [ ]:
# INRIX XD segments removed in this map release update
# (as compared to the previous map release)
xd_segments_removed_gdf = xd_segments_removed(
    previous_xd_segments_gdf, current_version
)
summary["num_segments_removed"] = len(xd_segments_removed_gdf)
xd_segments_removed_gdf

In [ ]:
print(summary)

In [ ]:
# all the below cells were copied from xd_diff_2201
# and have not really been looked at yet

In [ ]:
corr_df = pd.read_csv(network_conflation_correspondences_filepath)
count_series = corr_df['INRIX_SegID'].value_counts()

onetoone_series = count_series[count_series==1]
onetoone_series = onetoone_series.reset_index()

count_series = count_series[count_series>1]
count_series = count_series.reset_index()
corr_df2 = corr_df.drop_duplicates('INRIX_SegID')

In [ ]:
replace_list = replace_df['XDId_23_1'].tolist()

xd_rep_1 = onetoone_series.loc[onetoone_series['INRIX_SegID'].isin(replace_list), ]
print(f'{current_version} XD Segments Replaced with one-to-one Corr - {len(xd_rep_1)}')

xd_rep_2101 = corr_df2.loc[corr_df2['INRIX_SegID'].isin(replace_list), ]
print(f'{current_version} XD Segments Replaced - {len(xd_rep_2101)}')
replace_list = xd_rep_2101['INRIX_SegID'].tolist()
xd_rep_2101 = corr_df.loc[corr_df['INRIX_SegID'].isin(replace_list), ]
cmp_list = xd_rep_2101['CMP_SegID'].unique().tolist()
xd_rep_2101 = corr_df.loc[corr_df['CMP_SegID'].isin(cmp_list), ]
order_df = xd_rep_2101[['CMP_SegID','INRIX_SegID']]
xd_rep_2101 = xd_rep_2101.merge(replace_df[['XDId_22_1', 'XDId_23_1']], how='left', 
                                left_on='INRIX_SegID', right_on='XDId_22_1')

current_xd_segments_gdf['XDId_23_1'] = current_xd_segments_gdf['XDSegID'].astype(float)
xd_rep_2101 = xd_rep_2101.merge(current_xd_segments_gdf[['XDId_23_1','RoadName','Miles']], how='left', on='XDId_23_1')
xd_rep_2101 = order_df.merge(xd_rep_2101)
xd_rep_2101.to_csv(join(conflation_dir, 'XD_2201_replaced.csv'), index=False)

xd_rem_norep = xd_segments_removed_gdf.loc[~xd_segments_removed_gdf['XDSegID'].isin(replace_list), ]
print(f'XD Segments Removed but not Replaced - {len(xd_rem_norep)}')
previous_xd_segments_gdf['INRIX_SegID'] = previous_xd_segments_gdf['XDSegID'].astype(int)
xd_rem_norep = xd_rem_norep.merge(previous_xd_segments_gdf[['XDSegID','RoadName','Miles']], how='left', on='XDSegID')
xd_rem_norep.to_csv(join(conflation_dir, 'XD_2102_removed.csv'), index=False)